In [5]:
%pip install langchain

  Using cached langchain-0.0.300-py3-none-any.whl (1.7 MB)
  Using cached PyYAML-6.0.1-cp311-cp311-macosx_11_0_arm64.whl (167 kB)
  Using cached SQLAlchemy-2.0.21-cp311-cp311-macosx_11_0_arm64.whl (2.0 MB)
  Using cached aiohttp-3.8.5-cp311-cp311-macosx_11_0_arm64.whl (339 kB)
  Using cached anyio-3.7.1-py3-none-any.whl (80 kB)
  Using cached dataclasses_json-0.6.0-py3-none-any.whl (27 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langsmith-0.0.40-py3-none-any.whl (39 kB)
  Using cached numexpr-2.8.6-cp311-cp311-macosx_11_0_arm64.whl (91 kB)
  Using cached numpy-1.26.0-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Using cached pydantic-2.3.0-py3-none-any.whl (374 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-3.2.0-cp311-cp311-macosx_11_0_arm64.whl (122 kB)
  Using cached multidict-6.0.4-cp311-cp311-ma

In [ ]:
import DBObject
import sqlite3
import generatePrompt
from langchain.utilities import SQLDatabase
from typing import List

In [ ]:
def connect_fun(database_name: str) -> DBObject:
  conn = SQLDatabase.from_uri(f"sqlite://{database_name}?mode=ro") # open in readonly mode
  return DBObject(conn)

In [ ]:

def query_fun(question: str, tables: List[str], database_connection: DBObject) -> str:
  prompt = generatePrompt()

  pass